In [1]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

# Monte Carlo Simulation and Linear Algebra

**CS1302 Introduction to Computer Programming**
___

In [7]:
import math
import random
import numpy as np
import ipywidgets as widgets
%reload_ext divewidgets

## Monte Carlo simulation

**What is Monte Carlo simulation?**

 > The name Monte Carlo refers to the [Monte Carlo Casino in Monaco](https://en.wikipedia.org/wiki/Monte_Carlo_Casino) where Ulam's uncle would borrow money from relatives to gamble.

It would be nice to simulate the casino so Ulam's uncle did not need to borrow money to play in the casino.  
Actually...,
- Monte Carlo is the code name of the secret project for creating the [hydrogen bomb](https://en.wikipedia.org/wiki/Monte_Carlo_method). 
- [Ulam](https://en.wikipedia.org/wiki/Stanislaw_Ulam) worked with [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann) to program the first electronic computer ENIAC to simulate a computational model of a thermonuclear reaction.

(See also [The Beginning of the Monte Carlo Method](https://permalink.lanl.gov/object/tr?what=info:lanl-repo/lareport/LA-UR-88-9067) for a more detailed account.)

**How to compute the value of $\pi$**?

An application of Monte Carlo simulation is to approximate $\pi$ using 
the [Buffon's needle](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem).  
There is [a program](https://www.khanacademy.org/computer-programming/pi-by-buffons-needle/6695500989890560) written in javascript to do this.

The javascript program is long, so we will use an alternative simulation that is easier to understand/program.

If we uniformly randomly pick a point in a square. What is the chance the point is inside the inscribed circle, i.e., the biggest circle inside the square?

The chance is the circle's area divided by the square's area. Suppose the square has length $\ell$, then the chance is

$$ \frac{\pi (\ell /2)^2}{ (\ell)^2 } = \frac{\pi}4 $$
independent of the length $\ell$.

**Exercise** Complete the following function to return an approximation of $\pi$ as follows:
1. Simulate the random process of picking a point from a square repeatedly `n` times by  
  generating the $x$ and $y$ coordinates uniformly randomly from a unit interval $[0,1)$.
2. Compute the fraction of times the point is in the first quadrant of the inscribed circle, as shown in the figure below.
3. Return $4$ times the fraction as the approximation.
<p><a href="https://commons.wikimedia.org/wiki/File:Pi_30K.gif#/media/File:Pi_30K.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" alt="Pi 30K.gif"></a></p>

In [3]:
%%timeit -n 1 -r 1
def approximate_pi(n):
    ### BEGIN SOLUTION
    return (
        4
        * len(
            [True for i in range(n) if random.random() ** 2 + random.random() ** 2 < 1]
        )
        / n
    )
    ### END SOLUTION
print(f"Approximate: {approximate_pi(int(1e7))}\nGround truth: {math.pi}")

Approximate: 3.1417192
Ground truth: 3.141592653589793
2.95 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**How accurate is the approximation?**

The following uses a powerful library `numpy` for computing to return a [$95\%$-confidence interval](http://onlinestatbook.com/2/estimation/mean.html#:~:text=To%20compute%20the%2095%25%20confidence,be%20between%20the%20cutoff%20points.).

In [4]:
%%timeit -n 1 -r 1
import numpy as np


def np_approximate_pi(n):
    in_circle = (np.random.random((n, 2)) ** 2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n ** 0.5
    return np.array([mean - 2 * std, mean + 2 * std])


interval = np_approximate_pi(int(1e7))
print(
    f"""95%-confidence interval: {interval}
Estimate: {interval.mean():.4f} ± {(interval[1]-interval[0])/2:.4f}
Ground truth: {math.pi}"""
)

95%-confidence interval: [3.14003516 3.14211284]
Estimate: 3.1411 ± 0.0010
Ground truth: 3.141592653589793
756 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


```{note}

The computation done using `numpy` is over $5$ times faster despite the additional computation of the standard deviation.
```

```{seealso}

There are faster methods to approximate $\pi$, such as the [Chudnovsky_algorithm](https://en.wikipedia.org/wiki/Chudnovsky_algorithm), but the Monte-Carlo method is helpful in complicated situations.   
E.g., see the Monte Carlo simulation of a [real-life situation](https://www.youtube.com/watch?v=-fCVxTTAtFQ) in playing basketball:
> "When down by three and left with only 30 seconds, is it better to attempt a hard 3-point shot or an easy 2-point shot and get another possession?"   --LeBron James
```

## Linear Algebra

**How to solve a linear equation?**

Given the following linear equation in variable $x$ with real-valued coefficient $a$ and $b$,

$$ a x = b,$$
what is the value of $x$ that satisfies the equation?

**Exercise** Complete the following function to return either the unique solution of $x$ or `None` if a unique solution does not exist.

In [5]:
def solve_linear_equation(a, b):
    ### BEGIN SOLUTION
    return b / a if a != 0 else None
    ### END SOLUTION


@widgets.interact(a=(0, 5, 1), b=(0, 5, 1))
def linear_equation_solver(a=2, b=3):
    print(
        f"""linear equation: {a}x = {b}
       solution: x = {solve_linear_equation(a,b)}"""
    )

interactive(children=(IntSlider(value=2, description='a', max=5), IntSlider(value=3, description='b', max=5), …

**How to solve multiple linear equations?**

In the general case, we have a system of $m$ linear equations and $n$ variables:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 + \dots + a_{0(n-1)} x_{n-1} &= b_0\\
a_{10} x_0 + a_{11} x_1 + \dots + a_{1(n-1)} x_{n-1} &= b_1\\
 & \vdots\\
a_{(m-1)0} x_0 + a_{(m-1)1} x_1 + \dots + a_{(m-1)(n-1)} x_{n-1} &= b_{m-1}\\
\end{aligned}
$$
where
- $x_j$ for $j\in \{0,\dots,n-1\}$ are the variables, and
- $a_{ij}$ and $b_j$ for $i\in \{0,\dots,m-1\}$ and $j\in \{0,\dots,n-1\}$ are the coefficients.

A fundamental problem in linear algebra is to compute the unique solution to the system if it exists.

We will consider the simpler 2-by-2 system with 2 variables and 2 equations:

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1.
\end{aligned}
$$

To get an idea of the solution, suppose 

$$a_{00}=a_{11}=1, a_{01} = a_{10} = 0.$$
The system of equations becomes

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{x_0 +}  x_1 &= b_1,
\end{aligned}
$$
which gives the solution directly.

What about $a_{00}=a_{11}=2$ instead?

$$ \begin{aligned}
2x_0 \hphantom{+ x_1} &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1,
\end{aligned}$$

To obtain the solution, we divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2.
\end{aligned}
$$

What if $a_{01}=2$ instead?

$$ \begin{aligned}
2x_0 + 2x_1 &= b_0\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$

The second equation gives the solution of $x_1$, and we can use the solution in the first equation to solve for $x_0$. More precisely:
- Subtract the second equation from the first one:

$$ \begin{aligned}
2x_0 \hphantom{+2x_1} &= b_0 - b_1\\
\hphantom{2x_0 +}  2x_1 &= b_1\\
\end{aligned}
$$
- Divide both equations by 2:

$$ \begin{aligned}
x_0 \hphantom{+ x_1} &= \frac{b_0 - b_1}2\\
\hphantom{x_0 +}  x_1 &= \frac{b_1}2\\
\end{aligned}
$$

```{important}

The above operations are called *row operations* in linear algebra: each row is an equation.  
A system of linear equations can be solved by the linear operations of 
1. multiplying an equation by a constant, and
2. subtracting one equation from another.
```

How to write a program to solve a general 2-by-2 system? We will use the `numpy` library.

### Creating `numpy` arrays

**How to store the coefficients?**

In linear algebra, a system of equations such as

$$ \begin{aligned}
a_{00} x_0 + a_{01} x_1 &= b_0\\
a_{10} x_0 + a_{11} x_1 &= b_1
\end{aligned}
$$
is written concisely in *matrix* form as $ \mathbf{A} \mathbf{x} = \mathbf{b} $:

$$\overbrace{\begin{bmatrix}
a_{00} & a_{01}\\
a_{10} & a_{11}
\end{bmatrix}}^{\mathbf{A}}
\overbrace{
\begin{bmatrix}
x_0\\
x_1
\end{bmatrix}}
^{\mathbf{x}}
= \overbrace{\begin{bmatrix}
b_0\\
b_1
\end{bmatrix}}^{\mathbf{b}},
$$
where
$ \mathbf{A} \mathbf{x}$ is the *matrix multiplication*

$$ \mathbf{A} \mathbf{x} = \begin{bmatrix}
a_{00} x_0 + a_{01} x_1\\
a_{10} x_0 + a_{11} x_1
\end{bmatrix}.
$$

We say that $\mathbf{A}$ is a [*matrix*](https://en.wikipedia.org/wiki/Matrix_(mathematics)) and its dimension/shape is $2$-by-$2$:
- The first dimension/axis has size $2$. We also say that the matrix has $2$ rows.
- The second dimension/axis has size $2$. We also say that the matrix has $2$ columns.
$\mathbf{x}$ and $\mathbf{b}$ are called column vectors, which are matrices with one column.

Consider the example

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1,
\end{aligned}$$

or in matrix form with

$$ \begin{aligned}
\mathbf{A}&=\begin{bmatrix}
a_{00} & a_{01} \\
a_{10} & a_{11} 
\end{bmatrix} 
= \begin{bmatrix}
2 & 2 \\
0 & 2 
\end{bmatrix}\\
\mathbf{b}&=\begin{bmatrix}
b_0\\
b_1
\end{bmatrix} = \begin{bmatrix}
1\\
1
\end{bmatrix}\end{aligned}$$

Instead of using `list` to store the matrix, we will use a `numpy` array.

In [8]:
A = np.array([[2.0, 2], [0, 2]])
b = np.array([1.0, 1])
A, b

(array([[2., 2.],
        [0., 2.]]),
 array([1., 1.]))

Compared to `list`, `numpy` array is often more efficient and has more attributes.

In [9]:
array_attributes = set(attr for attr in dir(np.array([])) if attr[0] != "_")
list_attributes = set(attr for attr in dir(list) if attr[0] != "_")
print("\nCommon attributes:\n", *(array_attributes & list_attributes))
print("\nArray-specific attributes:\n", *(array_attributes - list_attributes))
print("\nList-specific attributes:\n", *(list_attributes - array_attributes))


Common attributes:
 sort copy

Array-specific attributes:
 tolist fill newbyteorder transpose item argmin all take setflags base put max nbytes flags argpartition ptp setfield tostring data dump dot sum ravel strides astype searchsorted partition dtype repeat flatten ndim mean round clip real trace conjugate dumps view argmax shape byteswap prod min cumprod ctypes choose imag getfield conj T squeeze std reshape resize tofile cumsum swapaxes compress size itemset diagonal var flat itemsize any argsort nonzero tobytes

List-specific attributes:
 remove pop append count reverse index insert clear extend


The following attributes give the dimension/shape, number of dimensions, size, and data type.

In [10]:
for array in A, b:
    print(
        f"""{array}
    shape: {array.shape}
    ndim: {array.ndim}
    size: {array.size}
    dtype: {array.dtype}
    """
    )

[[2. 2.]
 [0. 2.]]
    shape: (2, 2)
    ndim: 2
    size: 4
    dtype: float64
    
[1. 1.]
    shape: (2,)
    ndim: 1
    size: 2
    dtype: float64
    


Note that the function `len` only returns the size of the first dimension:

In [11]:
assert A.shape[0] == len(A)
len(A)

2

Unlike `list`, every `numpy` array has a data type. For efficient computation/storage, numpy implements different data types with different storage sizes:
* integer: `int8`, `int16`, `int32`, `int64`, `uint8`, ...
* float: `float16`, `float32`, `float64`, ...
* complex: `complex64`, `complex128`, ...
* boolean: `bool8`
* Unicode: `string`
* Object: `object`

E.g., `int64` is the 64-bit integer. Unlike `int`, `int64` has a range.

In [12]:
%%optlite -h 400
import numpy as np
min_int64 = np.int64(-2**63)
max_int64 = np.int64(2**63-1)
np.int64(2 ** 63)  # overflow error

OPTWidget(value=None, height=400, script='import numpy as np\nmin_int64 = np.int64(-2**63)\nmax_int64 = np.int…

We can use the `astype` method to convert the data type:

In [13]:
A_int64 = A.astype(int)  # converts to int64 by default
A_float32 = A.astype(np.float32)  # converts to float32
for array in A_int64, A_float32:
    print(array, array.dtype)

[[2 2]
 [0 2]] int64
[[2. 2.]
 [0. 2.]] float32


We must be careful about assigning items of different types to an array.

In [14]:
A_int64[0, 0] = 1
print(A_int64)
A_int64[0, 0] = 0.5
print(A_int64)  # intended assignment fails
np.array([int(1), float(1)])  # will be all floating point numbers

[[1 2]
 [0 2]]
[[0 2]
 [0 2]]


array([1., 1.])

**Exercise** Create a heterogeneous `numpy` array to store both integers and strings:
```Python
[0, 1, 2, 'a', 'b', 'c']
```
```{hint}
There is an `numpy` data type called `object`.
```

In [15]:
np.object?
### BEGIN SOLUTION
heterogeneous_np_array = np.array([*range(3), *"abc"], dtype=object)
### END SOLUTION
heterogeneous_np_array

array([0, 1, 2, 'a', 'b', 'c'], dtype=object)

Init signature: np.object()
Docstring:     
The base class of the class hierarchy.

When called, it accepts no arguments and returns a new featureless
instance that has no instance attributes and cannot be given any.
Type:           type
Subclasses:     type, async_generator, int, bytearray_iterator, bytearray, bytes_iterator, bytes, builtin_function_or_method, callable_iterator, PyCapsule, ...


Be careful when creating arrays of `tuple`/`list`:

In [16]:
%%optlite -h 350
import numpy as np
a1 = np.array([(1, 2), [3, 4, 5]], dtype=object)
print(a1.shape, a1.size)
a2 = np.array([(1, 2), [3, 4]], dtype=object)
print(a2.shape, a2.size)

OPTWidget(value=None, height=350, script='import numpy as np\na1 = np.array([(1, 2), [3, 4, 5]], dtype=object)…

`numpy` provides many functions to create an array:

In [17]:
np.zeros?

Docstring:
zeros(shape, dtype=float, order='C', *, like=None)

Return a new array of given shape and type, filled with zeros.

Parameters
----------
shape : int or tuple of ints
    Shape of the new array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    The desired data-type for the array, e.g., `numpy.int8`.  Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: 'C'
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    NumPy arrays. If an array-like passed in as ``like`` supports
    the ``__array_function__`` protocol, the result will be defined
    by it. In this case, it ensures the creation of an array object
    compatible with that passed in via this argument.

    .. versionadded:: 1.20.0

Returns
-------
out : ndarray
    Array of zeros with the given shape, dtype, and order.

See Al

In [18]:
%%optlite -h 400 -l
import numpy as np
a = [np.zeros(0)]
a.append(np.zeros(1, dtype=int))
a.append(np.zeros((2, 3, 4)))

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.zeros(0)]\na.append(np.zeros(1, dtype=in…

In [19]:
np.empty?

Docstring:
empty(shape, dtype=float, order='C', *, like=None)

Return a new array of given shape and type, without initializing entries.

Parameters
----------
shape : int or tuple of int
    Shape of the empty array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    Desired output data-type for the array, e.g, `numpy.int8`. Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: 'C'
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    NumPy arrays. If an array-like passed in as ``like`` supports
    the ``__array_function__`` protocol, the result will be defined
    by it. In this case, it ensures the creation of an array object
    compatible with that passed in via this argument.

    .. versionadded:: 1.20.0

Returns
-------
out : ndarray
    Array of uninitialized (arbitrary) data of the gi

In [20]:
%%optlite -h 400 -l
import numpy as np
a = [np.empty(2)]
a.append(np.empty((2, 3, 4), 
                  dtype=int))

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.empty(2)]\na.append(np.empty((2, 3, 4), …

In [21]:
np.ones?

Signature: np.ones(shape, dtype=None, order='C', *, like=None)
Docstring:
Return a new array of given shape and type, filled with ones.

Parameters
----------
shape : int or sequence of ints
    Shape of the new array, e.g., ``(2, 3)`` or ``2``.
dtype : data-type, optional
    The desired data-type for the array, e.g., `numpy.int8`.  Default is
    `numpy.float64`.
order : {'C', 'F'}, optional, default: C
    Whether to store multi-dimensional data in row-major
    (C-style) or column-major (Fortran-style) order in
    memory.
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    NumPy arrays. If an array-like passed in as ``like`` supports
    the ``__array_function__`` protocol, the result will be defined
    by it. In this case, it ensures the creation of an array object
    compatible with that passed in via this argument.

    .. versionadded:: 1.20.0

Returns
-------
out : ndarray
    Array of ones with the given shape, dtype, and orde

In [22]:
%%optlite -h 400 -l
import numpy as np
a = [np.ones(2)]
a.append(np.ones((2, 3, 4),
                dtype=int))

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.ones(2)]\na.append(np.ones((2, 3, 4),\n …

In [23]:
np.eye?

Signature: np.eye(N, M=None, k=0, dtype=<class 'float'>, order='C', *, like=None)
Docstring:
Return a 2-D array with ones on the diagonal and zeros elsewhere.

Parameters
----------
N : int
  Number of rows in the output.
M : int, optional
  Number of columns in the output. If None, defaults to `N`.
k : int, optional
  Index of the diagonal: 0 (the default) refers to the main diagonal,
  a positive value refers to an upper diagonal, and a negative value
  to a lower diagonal.
dtype : data-type, optional
  Data-type of the returned array.
order : {'C', 'F'}, optional
    Whether the output should be stored in row-major (C-style) or
    column-major (Fortran-style) order in memory.

    .. versionadded:: 1.14.0
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    NumPy arrays. If an array-like passed in as ``like`` supports
    the ``__array_function__`` protocol, the result will be defined
    by it. In this case, it ensures the creation of 

In [24]:
%%optlite -l -h 400
import numpy as np
a = [np.eye(2)]
a.append(np.eye(3, dtype=int))

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.eye(2)]\na.append(np.eye(3, dtype=int))\…

In [25]:
np.diag?

Signature: np.diag(v, k=0)
Docstring:
Extract a diagonal or construct a diagonal array.

See the more detailed documentation for ``numpy.diagonal`` if you use this
function to extract a diagonal and wish to write to the resulting array;
whether it returns a copy or a view depends on what version of numpy you
are using.

Parameters
----------
v : array_like
    If `v` is a 2-D array, return a copy of its `k`-th diagonal.
    If `v` is a 1-D array, return a 2-D array with `v` on the `k`-th
    diagonal.
k : int, optional
    Diagonal in question. The default is 0. Use `k>0` for diagonals
    above the main diagonal, and `k<0` for diagonals below the main
    diagonal.

Returns
-------
out : ndarray
    The extracted diagonal or constructed diagonal array.

See Also
--------
diagonal : Return specified diagonals.
diagflat : Create a 2-D array with the flattened input as a diagonal.
trace : Sum along diagonals.
triu : Upper triangle of an array.
tril : Lower triangle of an array.

Examples

In [26]:
%%optlite -l -h 400
import numpy as np
a = [np.diag(range(1))]
a.append(np.diag(range(2)))
a.append(np.diag(np.ones(3), k=1))
# 1 above main diagonal

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.diag(range(1))]\na.append(np.diag(range(…

`numpy` also provides functions to build an array using rules.

In [27]:
np.arange?

Docstring:
arange([start,] stop[, step,], dtype=None, *, like=None)

Return evenly spaced values within a given interval.

``arange`` can be called with a varying number of positional arguments:

* ``arange(stop)``: Values are generated within the half-open interval
  ``[0, stop)`` (in other words, the interval including `start` but
  excluding `stop`).
* ``arange(start, stop)``: Values are generated within the half-open
  interval ``[start, stop)``.
* ``arange(start, stop, step)`` Values are generated within the half-open
  interval ``[start, stop)``, with spacing between values given by
  ``step``.

For integer arguments the function is roughly equivalent to the Python
built-in :py:class:`range`, but returns an ndarray rather than a ``range``
instance.

When using a non-integer step, such as 0.1, it is often better to use
`numpy.linspace`.

See the Warning sections below for more information.

Parameters
----------
start : integer or real, optional
    Start of interval.  The interva

In [28]:
%%optlite -l -h 400
import numpy as np
a = [np.arange(5)]
# like range but allow non-integer parameters
a.append(np.arange(2.5, 5, 0.5))

OPTWidget(value=None, height=400, script='import numpy as np\na = [np.arange(5)]\n# like range but allow non-i…

In [29]:
np.linspace?

Signature:
np.linspace(
    start,
    stop,
    num=50,
    endpoint=True,
    retstep=False,
    dtype=None,
    axis=0,
)
Docstring:
Return evenly spaced numbers over a specified interval.

Returns `num` evenly spaced samples, calculated over the
interval [`start`, `stop`].

The endpoint of the interval can optionally be excluded.

.. versionchanged:: 1.16.0
    Non-scalar `start` and `stop` are now supported.

.. versionchanged:: 1.20.0
    Values are rounded towards ``-inf`` instead of ``0`` when an
    integer ``dtype`` is specified. The old behavior can
    still be obtained with ``np.linspace(start, stop, num).astype(int)``

Parameters
----------
start : array_like
    The starting value of the sequence.
stop : array_like
    The end value of the sequence, unless `endpoint` is set to False.
    In that case, the sequence consists of all but the last of ``num + 1``
    evenly spaced samples, so that `stop` is excluded.  Note that the step
    size changes when `endpoint` is Fals

In [30]:
%%optlite -l -h 400
import numpy as np
# can specify the number of points instead of the step size
a = [np.linspace(0, 4, 5)]
a.append(np.linspace(2.5, 4.5, 5))

OPTWidget(value=None, height=400, script='import numpy as np\n# can specify number of points instead of step\n…

In [31]:
np.fromfunction?

Signature:
np.fromfunction(
    function,
    shape,
    *,
    dtype=<class 'float'>,
    like=None,
    **kwargs,
)
Docstring:
Construct an array by executing a function over each coordinate.

The resulting array therefore has a value ``fn(x, y, z)`` at
coordinate ``(x, y, z)``.

Parameters
----------
function : callable
    The function is called with N parameters, where N is the rank of
    `shape`.  Each parameter represents the coordinates of the array
    varying along a specific axis.  For example, if `shape`
    were ``(2, 2)``, then the parameters would be
    ``array([[0, 0], [1, 1]])`` and ``array([[0, 1], [0, 1]])``
shape : (N,) tuple of ints
    Shape of the output array, which also determines the shape of
    the coordinate arrays passed to `function`.
dtype : data-type, optional
    Data-type of the coordinate arrays passed to `function`.
    By default, `dtype` is float.
like : array_like, optional
    Reference object to allow the creation of arrays which are not
    

In [32]:
%%optlite -l -h 400
import numpy as np
a = np.fromfunction(lambda i, j: i * j, (3, 4))

OPTWidget(value=None, height=400, script='import numpy as np\na = np.fromfunction(lambda i, j: i * j, (3, 4))\…

We can also reshape an array using the `reshape` method/function:

In [33]:
np.ndarray.reshape?

Docstring:
a.reshape(shape, order='C')

Returns an array containing the same data with a new shape.

Refer to `numpy.reshape` for full documentation.

See Also
--------
numpy.reshape : equivalent function

Notes
-----
Unlike the free function `numpy.reshape`, this method on `ndarray` allows
the elements of the shape parameter to be passed in as separate arguments.
For example, ``a.reshape(10, 11)`` is equivalent to
``a.reshape((10, 11))``.
Type:      method_descriptor


In [34]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
c1 = a.reshape(2, 3, 4)
# automatically calculate the size specified as `-1`
c2 = a.reshape(2, 3, -1)

OPTWidget(value=None, height=500, script='import numpy as np\na = np.arange(2 * 3 * 4)\nc1 = a.reshape(2, 3, 4…

In [35]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# default C ordering
c = a.reshape((2, 3, -1), order="C")
# F ordering
f = a.reshape((2, 3, -1), order="F")

OPTWidget(value=None, height=500, script='import numpy as np\na = np.arange(2 * 3 * 4)\n# default C ordering\n…

The C ordering and F ordering can be implemented using list comprehension:

In [36]:
%%optlite -l -h 500
import numpy as np
a = np.arange(2 * 3 * 4)
# last axis index changes fastest
c = np.array(
    [[[a[i*3*4+j*4+k] 
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])
# first axis index changes fastest
f = np.array(
    [[[a[i+j*2+k*3*2]
       for k in range(4)]
      for j in range(3)]
     for i in range(2)])

OPTWidget(value=None, height=500, script='import numpy as np\na = np.arange(2 * 3 * 4)\n# last axis index chan…

`ravel` is a particular case of reshaping an array to one dimension.  
A similar function `flatten` returns a copy of the array but `ravel` and `reshape` returns a dynamic view whenever possible.

In [37]:
array = np.arange(2 * 3 * 4).reshape(2, 3, 4)
array, array.ravel(), array.reshape(-1), array.ravel(order="F")

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 array([ 0, 12,  4, 16,  8, 20,  1, 13,  5, 17,  9, 21,  2, 14,  6, 18, 10,
        22,  3, 15,  7, 19, 11, 23]))

**Exercise** Correct the following function to print every element of an array line-by-line.
```Python
def print_array_entries_line_by_line(array):
    for i in array:
        print(i)
```

In [38]:
def print_array_entries_line_by_line(array):
    ### BEGIN SOLUTION
    for i in array.flatten():
        print(i)
    ### END SOLUTION


print_array_entries_line_by_line(np.arange(2 * 3 * 4).reshape(2, 3, 4))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


### Operating on `numpy` arrays

**How to verify the solution of a system of linear equations?**

Before solving the system of linear equations, let us try to verify a solution to the equations:

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1
\end{aligned}
$$

`numpy` provides the function `matmul` and the operator `@` for matrix multiplication.

In [41]:
print(np.matmul(A, np.array([0, 0])) == b)
print(A @ np.array([0, 0.5]) == b)

[False False]
[ True  True]


Note that the comparison on `numpy` arrays returns a boolean array instead of a boolean value, unlike the comparison operations on lists.

To check whether all items are true, we use the `all` method.

In [42]:
print((np.matmul(A, np.array([0, 0])) == b).all())
print((A @ np.array([0, 0.5]) == b).all())

False
True


**How to concatenate arrays?**

We will operate on an augmented matrix of the coefficients:

$$ \begin{aligned} \mathbf{C} &= \begin{bmatrix}
\mathbf{A} & \mathbf{b}
\end{bmatrix}\\
&= \begin{bmatrix}
a_{00} & a_{01} & b_0 \\
a_{10} & a_{11} & b_1
\end{bmatrix} 
\end{aligned}
$$

`numpy` provides functions to create block matrices:

In [43]:
np.block?
C = np.block([A, b.reshape(-1, 1)])  # reshape to ensure same ndim
C

array([[2., 2., 1.],
       [0., 2., 1.]])

Signature: np.block(arrays)
Docstring:
Assemble an nd-array from nested lists of blocks.

Blocks in the innermost lists are concatenated (see `concatenate`) along
the last dimension (-1), then these are concatenated along the
second-last dimension (-2), and so on until the outermost list is reached.

Blocks can be of any dimension, but will not be broadcasted using the normal
rules. Instead, leading axes of size 1 are inserted, to make ``block.ndim``
the same for all blocks. This is primarily useful for working with scalars,
and means that code like ``np.block([v, 1])`` is valid, where
``v.ndim == 1``.

When the nested list is two levels deep, this allows block matrices to be
constructed from their components.

.. versionadded:: 1.13.0

Parameters
----------
arrays : nested list of array_like or scalars (but not tuples)
    If passed a single ndarray or scalar (a nested list of depth 0), this
    is returned unmodified (and not copied).

    Elements shapes must match along the appropr

To stack an array along different axes:

In [44]:
array = np.arange(1 * 2 * 3).reshape(1, 2, 3)
for concat_array in [
    array,
    np.hstack((array, array)),  # stack along the first axis
    np.vstack((array, array)),  # second axis
    np.concatenate((array, array), axis=-1),  # last axis
    np.stack((array, array), axis=0),
]:  # new axis
    print(concat_array, "\nshape:", concat_array.shape)

[[[0 1 2]
  [3 4 5]]] 
shape: (1, 2, 3)
[[[0 1 2]
  [3 4 5]
  [0 1 2]
  [3 4 5]]] 
shape: (1, 4, 3)
[[[0 1 2]
  [3 4 5]]

 [[0 1 2]
  [3 4 5]]] 
shape: (2, 2, 3)
[[[0 1 2 0 1 2]
  [3 4 5 3 4 5]]] 
shape: (1, 2, 6)
[[[[0 1 2]
   [3 4 5]]]


 [[[0 1 2]
   [3 4 5]]]] 
shape: (2, 1, 2, 3)


**How to perform arithmetic operations on a `numpy` array?**

To divide all the coefficients by $2$, we can write:

In [45]:
D = C / 2
D

array([[1. , 1. , 0.5],
       [0. , 1. , 0.5]])

Note that the above does not work for `list`.

In [61]:
%%optlite -l -h 500
[[2, 2, 1], [0, 2, 1]] / 2

OPTWidget(value=None, height=500, script='[[2, 2, 1], [0, 2, 1]] / 2\n', srcprefix='https://dive4dec.github.io…

Arithmetic operations on `numpy` arrays apply if the arrays have compatible dimensions. Two dimensions are compatible when
- they are equal, except for
- components equal to 1.

`numpy` uses [broadcasting rules](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules) to stretch the axis of size 1 up to match the corresponding axis in other arrays.  
`C / 2` is an example where the second operand $2$ is broadcasted to a $2$-by-$2$ matrix before the elementwise division. Another example is as follows.

In [47]:
three_by_one = np.arange(3).reshape(3, 1)
one_by_four = np.arange(4).reshape(1, 4)
print(
    f"""
{three_by_one}
*
{one_by_four}
==
{three_by_one * one_by_four}
"""
)


[[0]
 [1]
 [2]]
*
[[0 1 2 3]]
==
[[0 0 0 0]
 [0 1 2 3]
 [0 2 4 6]]



Next, to subtract the second row of the coefficients from the first row:

In [48]:
D[0, :] = D[0, :] - D[1, :]
D

array([[1. , 0. , 0. ],
       [0. , 1. , 0.5]])

Notice the use of commas to index different dimensions instead of using multiple brackets:

In [49]:
assert (D[0][:] == D[0, :]).all()

Using this indexing technique, it is easy to extract the last column as the solution to the system of linear equations:

In [50]:
x = D[:, -1]
x

array([0. , 0.5])

This gives the desired solution $x_0=0$ and $x_1=0.5$ for

$$ \begin{aligned}
2x_0 + 2x_1 &= 1\\
\hphantom{2x_0 +}  2x_1 &= 1\\
\end{aligned}$$

`numpy` provides many [convenient ways](https://numpy.org/doc/stable/reference/arrays.indexing.html#advanced-indexing) to index an array.

In [51]:
B = np.arange(2 * 3).reshape(2, 3)
B, B[(0, 1), (2, 0)]  # selecting the corners using integer array

(array([[0, 1, 2],
        [3, 4, 5]]),
 array([2, 3]))

In [52]:
B = np.arange(2 * 3 * 4).reshape(2, 3, 4)
B, B[0], B[0, (1, 2)], B[0, (1, 2), (2, 3)], B[
    :, (1, 2), (2, 3)
]  # pay attention to the last two cases

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([ 6, 11]),
 array([[ 6, 11],
        [18, 23]]))

In [53]:
assert (B[..., -1] == B[:, :, -1]).all()
B[..., -1]  # ... expands to selecting all elements of all previous dimensions

array([[ 3,  7, 11],
       [15, 19, 23]])

In [54]:
B[B > 5]  # indexing using boolean array

array([ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23])

Finally, the following function solves 2 linear equations with 2 variables.

In [55]:
def solve_2_by_2_system(A, b):
    """Returns the unique solution of the linear system, if exists,
    else returns None."""
    C = np.hstack((A, b.reshape(-1, 1)))
    if C[0, 0] == 0:
        C = C[(1, 0), :]
    if C[0, 0] == 0:
        return None
    C[0, :] = C[0, :] / C[0, 0]
    C[1, :] = C[1, :] - C[0, :] * C[1, 0]
    if C[1, 1] == 0:
        return None
    C[1, :] = C[1, :] / C[1, 1]
    C[0, :] = C[0, :] - C[1, :] * C[0, 1]
    return C[:, -1]

In [56]:
# tests
for A in (
    np.eye(2),
    np.ones((2, 2)),
    np.stack((np.ones(2), np.zeros(2))),
    np.stack((np.ones(2), np.zeros(2)), axis=1),
):
    print(f"A={A}\nb={b}\nx={solve_2_by_2_system(A,b)}\n")

A=[[1. 0.]
 [0. 1.]]
b=[1. 1.]
x=[1. 1.]

A=[[1. 1.]
 [1. 1.]]
b=[1. 1.]
x=None

A=[[1. 1.]
 [0. 0.]]
b=[1. 1.]
x=None

A=[[1. 0.]
 [1. 0.]]
b=[1. 1.]
x=None



### Universal functions

Why does the first line of code below return two arrays but the second code return only one array? Shouldn't the first line of code return the following?
```Python
array([[(0,1), (0,2), (0,3)],
       [(1,1), (1,2), (1,3)]])
```

In [57]:
print(np.fromfunction(lambda i, j: (i, j), (2, 3), dtype=int))
print(np.fromfunction(lambda i, j: (i * j), (2, 3), dtype=int))

(array([[0, 0, 0],
       [1, 1, 1]]), array([[0, 1, 2],
       [0, 1, 2]]))
[[0 0 0]
 [0 1 2]]


From the documentation, `fromfunction` applies the given function to the two arrays as arguments.
- The first line of code returns a tuple of the arrays.
- The second line of code multiplies the two arrays to give one array, according to how multiplication works for numpy arrays.

Indeed, `numpy` implements [universal/vectorized functions/operators](https://numpy.org/doc/stable/reference/ufuncs.html) that take arrays as arguments and perform operations with appropriate broadcasting rules. The following is an example that uses the universal function `np.sin`:

In [58]:
import matplotlib.pyplot as plt


@widgets.interact(a=(0, 5, 1), b=(-1, 1, 0.1))
def plot_sin(a=1, b=0):
    x = np.linspace(0, 2 * math.pi)
    plt.plot(x, np.sin(a * x + b * math.pi))  # np.sin, *, + are universal functions
    plt.title(r"$\sin(ax+b\pi)$")
    plt.xlabel(r"$x$ (radian)")

interactive(children=(IntSlider(value=1, description='a', max=5), FloatSlider(value=0.0, description='b', max=…

In addition to making the code shorter, universal functions are both efficient and flexible. (Recall the Monte Carlo simulation to approximate $\pi$.)

**Exercise** Explain how the Monte Carlo simulation work using universal functions:
```Python
def np_approximate_pi(n):
    in_circle = (np.random.random((n,2))**2).sum(axis=-1) < 1
    mean = 4 * in_circle.mean()
    std = 4 * in_circle.std() / n**0.5
    return np.array([mean - 2*std, mean + 2*std])
```

```{admonition} Solution

- `random.random` generates a numpy array for $n$ points in the unit square randomly.
- `sum` sums up the element along the last axis to give the squared distance.
- `<` returns the boolean array indicating whether each point is in the first quadrant of the inscribed circle.
- `mean` and `std` returns the mean and standard deviation of the boolean array with True and False interpreted as 1 and 0, respectively.
```